In [2]:
# Import our dependencies
import pandas as pd
import matplotlib as plt
import numpy as np
import math
import geopy

In [3]:
# define column names for text file mapping
colNames = ['geonameid','name','asciiname','alternatenames',
           'latitude','longitude','feature class', 'feature code',
           'country code', 'cc2', 'admin1 code', 'admin2 code',
           'admin3 code', 'admin4 code', 'population', 'elevation',
           'dem', 'timezone', 'modification_date']

In [4]:
# load supplemental data into pandas
pop_df = pd.read_table('./Resources/population_data/TZ.txt', header=None)
pop_df.columns = colNames
pop_df.sort_values(by='population', ascending=False).head()

,geonameid,name,asciiname,alternatenames,latitude,longitude,feature class,feature code,country code,cc2,admin1 code,admin2 code,admin3 code,admin4 code,population,elevation,dem,timezone,modification_date
930,149590,United Republic of Tanzania,United Republic of Tanzania,"An Tansain,An Tansáin,Orileede Tansania,Orílẹ́...",-6.00000,35.00000,A,PCLI,TZ,NaN,0.0,NaN,NaN,NaN,56318348,NaN,829,Africa/Dar_es_Salaam,2020-03-29
11530,160260,Dar es Salaam Region,Dar es Salaam Region,"Dar es Salaam,Dar es Salaam Region,Dar-es-Salaam",-6.83523,39.19597,A,ADM1,TZ,NaN,23.0,NaN,NaN,NaN,2791063,NaN,62,Africa/Dar_es_Salaam,2011-03-17
3545,152219,Mwanza Region,Mwanza Region,"Mkoa wa Mwanza,Mwanza,Mwanza Region",-2.75000,33.08333,A,ADM1,TZ,NaN,12.0,NaN,NaN,NaN,2772509,NaN,1206,Africa/Dar_es_Salaam,2015-05-04
5683,154375,Mbeya Region,Mbeya Region,"Mbeya,Mbeya Region,Mkoa wa Mbeya",-8.20000,33.33333,A,ADM1,TZ,NaN,9.0,NaN,NaN,NaN,2707410,NaN,1315,Africa/Dar_es_Salaam,2015-05-04
11533,160263,Dar es Salaam,Dar es Salaam,"DAR,Dar Es Salaam,Dar Es Salam,Dar es Salaam,D...",-6.82349,39.26951,P,PPLA,TZ,NaN,23.0,702.0,702132.0,NaN,2698652,NaN,24,Africa/Dar_es_Salaam,2019-09-05


In [39]:
# subset data to include just city level data (admin3 code)
city_pop_df = pop_df[pop_df['admin3 code'].notnull()]
city_pop_df = city_pop_df[city_pop_df['population']>0]
city_pop_df.sort_values(by='population', ascending=False).head()

,geonameid,name,asciiname,alternatenames,latitude,longitude,feature class,feature code,country code,cc2,admin1 code,admin2 code,admin3 code,admin4 code,population,elevation,dem,timezone,modification_date
11533,160263,Dar es Salaam,Dar es Salaam,"DAR,Dar Es Salaam,Dar Es Salam,Dar es Salaam,D...",-6.82349,39.26951,P,PPLA,TZ,NaN,23.0,702.0,702132.0,NaN,2698652,NaN,24,Africa/Dar_es_Salaam,2019-09-05
536,149193,Unguja Ukuu Kaebona,Unguja Ukuu Kaebona,"Unguja Ukuu,Unguja Ukuu Kaebona",-6.27452,39.37486,A,ADM3,TZ,NaN,21.0,5201.0,5201381.0,NaN,1060416,NaN,32,Africa/Dar_es_Salaam,2016-07-25
73,148726,Zanzibar Island,Zanzibar Island,"Ile de Zanzibar,Menuthias,Unguja,Unguja Island...",-6.15557,39.34170,T,ISL,TZ,NaN,21.0,5201.0,5201181.0,NaN,896721,NaN,39,Africa/Dar_es_Salaam,2019-04-13
3549,152224,Mwanza,Mwanza,"MWZ,Muansa,Muanza,Muvanzo,Mvanza,Mwamza,Mwansa...",-2.51667,32.90000,P,PPLA,TZ,NaN,12.0,1903.0,1903042.0,NaN,436801,NaN,1144,Africa/Dar_es_Salaam,2019-09-05
77,148730,Zanzibar,Zanzibar,"Ilu Zanzibar,Poli tis Zanzivaris,Sansibar,Sans...",-6.16394,39.19793,P,PPLA,TZ,NaN,25.0,5302.0,5302272.0,NaN,403658,NaN,14,Africa/Dar_es_Salaam,2019-09-05


In [41]:
city_pop_df['population'].count()

280

In [15]:
from geopy.geocoders import get_geocoder_for_service
get_geocoder_for_service("nominatim")

geopy.geocoders.osm.Nominatim

In [28]:
# use geopy to transform lat/lon coordinates into a geoobject
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="population_map")

test_coord = f"{city_pop_df.latitude[0]} , {city_pop_df.longitude[0]}"

location = geolocator.reverse(test_coord)

In [34]:
import sqlite3

# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("Tanzania_Water_Pump.db")
water_df = pd.read_sql_query("SELECT * from pump_it_up_training_set", con)

con.close()

# Verify that result of SQL query is stored in the dataframe
water_df.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,0,0.0,2012-11-13,Tasaf,0,TASAF,33.1258283,-5.118154,Mratibu,0,...,milky,milky,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump,non functional
1,1,0.0,2011-03-05,Shipo,1978,SHIPO,34.77071669,-9.395642,none,0,...,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump,functional
2,2,0.0,2011-03-27,Lvia,0,LVIA,36.11505595,-6.279268,Bombani,0,...,soft,good,insufficient,insufficient,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,functional
3,3,10.0,2013-06-03,Germany Republi,1639,CES,37.14743219,-3.187555,Area 7 Namba 5,0,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
4,4,0.0,2011-03-22,Cmsr,0,CMSR,36.16489341,-6.099289,Ezeleda,0,...,soft,good,dry,dry,shallow well,shallow well,groundwater,hand pump,hand pump,non functional
